<a href="https://colab.research.google.com/github/ishandahal/Fun_Projects/blob/master/Collab_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Looking at Collabrative filtering (Recomender systems)

In [ ]:
!pip install fastai -U

In [ ]:
from fastai.collab import *

In [ ]:
from fastai.vision.all import *
from fastai.tabular.all import *
path = untar_data(URLs.ML_100k)

In [ ]:
ratings = pd.read_csv(path / 'u.data', sep="\t", header=None)
ratings.columns = ["user", 'movie', 'rating', 'timestamp']
ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [ ]:
movies = pd.read_csv(path / 'u.item', delimiter='|', encoding='latin-1',
                     usecols=(0,1), names=('movie', 'title'), header=None)
movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [ ]:
ratings = ratings.merge(movies)
ratings.head()

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [ ]:
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=64)
dls.show_batch()

,user,title,rating
0,7,Stephen King's The Langoliers (1995),2
1,621,Super Mario Bros. (1993),2
2,823,Bananas (1971),5
3,798,Aladdin (1992),5
4,25,Dumbo (1941),3
5,49,"Room with a View, A (1986)",3
6,927,Species (1995),4
7,406,In the Name of the Father (1993),4
8,614,Dead Man Walking (1995),4
9,343,"Madness of King George, The (1994)",5


In [ ]:
dls.classes

{'title': (#1665) ['#na#',"'Til There Was You (1997)",'1-900 (1994)','101 Dalmatians (1996)','12 Angry Men (1957)','187 (1997)','2 Days in the Valley (1996)','20,000 Leagues Under the Sea (1954)','2001: A Space Odyssey (1968)','3 Ninjas: High Noon At Mega Mountain (1998)'...],
 'user': (#944) ['#na#',1,2,3,4,5,6,7,8,9...]}

In [ ]:
n_users = len(dls.classes['user'])
n_movies = len(dls.classes['title'])

user_factors = torch.randn((n_users,5))
movie_factors = torch.randn((n_movies,5))
user_factors.shape, movie_factors.shape

(torch.Size([944, 5]), torch.Size([1665, 5]))

In [ ]:
### Using One hot Encoded Vector as an index
one_hot_3 = one_hot(3, n_movies).float()
one_hot_3

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [ ]:
movie_factors.T @ (one_hot_3) 

tensor([-0.3245, -0.8238, -0.9068, -0.5508,  0.2758])

In [ ]:
movie_factors[3]

tensor([-0.3245, -0.8238, -0.9068, -0.5508,  0.2758])

### Embedding layer optimizes indexing 

In [ ]:
class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors):
        self.user_factors = Embedding(n_users, n_factors)
        self.movie_factors = Embedding(n_movies, n_factors)

    def forward(self, x):
        users = self.user_factors(x[:,0])
        movies = self.movie_factors(x[:,1])
        return (users * movies).sum(dim=1)

In [ ]:
x,y = dls.one_batch()
x.shape

torch.Size([64, 2])

In [ ]:
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())

In [ ]:
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,1.740834,1.374641,00:06
1,1.023633,1.043277,00:06
2,0.932468,0.960025,00:06
3,0.844374,0.891788,00:06
4,0.804196,0.881549,00:06


In [ ]:
#Model without the bias term
class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0,5.5)):
        self.user_factors = Embedding(n_users,n_factors)
        self.movie_factoes = Embedding(n_movies,n_factors)
        self.y_range = y_range
    
    def forward(self, x):
        users = self.user_factors(x[:,0])
        movies = self.movie_factoes(x[:,1])
        return sigmoid_range((users * movies).sum(dim=1), *self.y_range)

In [ ]:
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,1.004480,0.993913,00:06
1,0.867993,0.879735,00:06
2,0.695650,0.838283,00:06
3,0.544583,0.831651,00:06
4,0.451522,0.833230,00:06


In [ ]:
#Model with the bias
class DotProduct(Module):
    def __init__(self, n_movies, n_users, n_factors, y_range=(0,5.5)):
        self.user_factors = Embedding(n_users, n_factors)
        self.movie_factors = Embedding(n_movies, n_factors)
        self.user_bias = Embedding(n_users, 1)
        self.movie_bias = Embedding(n_movies, 1)
        self.y_range = y_range

    def forward(self, x):
        users = self.user_factors(x[:,0])
        movies = self.movie_factors(x[:,1])
        res = (users * movies).sum(dim=1, keepdim=True)
        res += self.user_bias(x[:,0]) + self.movie_bias(x[:,1])
        return sigmoid_range(res, *self.y_range)

In [ ]:
# model = DotProduct(n_movies, n_users, 50)
# learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(6, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.962911,0.963788,00:13
1,0.841069,0.888226,00:13
2,0.770890,0.844773,00:13
3,0.620263,0.825851,00:13
4,0.485572,0.825027,00:13
5,0.414518,0.826140,00:13


In [ ]:
## Creating an embedding module
class T(Module):
    def __init__(self): self.a = torch.ones(3)

L(T().parameters())

(#0) []

In [ ]:
##Wrapping the initialization with nn.Parameter so it starts keeping track of the gradients

class T(Module):
    def __init__(self): self.a = nn.Parameter(torch.ones(3))

L(T().parameters())

(#1) [Parameter containing:
tensor([1., 1., 1.], requires_grad=True)]

In [ ]:
class T(Module):
    def __init__(self): self.a = nn.Linear(1, 3, bias=False)

t = T()
L(t.parameters())

(#1) [Parameter containing:
tensor([[-0.8292],
        [-0.3039],
        [ 0.5984]], requires_grad=True)]

In [ ]:
type(t.a.weight)

torch.nn.parameter.Parameter

In [ ]:
# Creating a tensor parameter with random initialization
def create_params(size):
    return nn.Parameter(torch.zeros(*size).normal_(0,0.01))

In [ ]:
class DotProductBias(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0,5.5)):
        self.user_factors = create_params((n_users, n_factors))
        self.movie_factors = create_params((n_movies, n_factors))
        self.user_bias = create_params((n_users, 1))
        self.movie_bias = create_params((n_movies, 1))
        self.y_range = y_range
    
    def forward(self, x):
        users = self.user_factors[x[:,0]]
        movies = self.movie_factors[x[:,1]]
        res = (users * movies).sum(dim=1, keepdim=True, )
        res += self.user_bias[x[:,0]] + self.movie_bias[x[:,1]]
        return sigmoid_range(res, *self.y_range)

In [ ]:
model = DotProductBias(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.920313,0.948918,00:12
1,0.843523,0.871374,00:12
2,0.725452,0.829907,00:12
3,0.589886,0.821868,00:12
4,0.486085,0.822764,00:12


In [ ]:
## Model imterpretation: Movies with highest bias 
movies_bias = learn.model.movie_bias.squeeze()
idxs = movies_bias.argsort()[-5:]
[dls.classes['title'][i] for i in idxs]

['Star Wars (1977)',
 "Schindler's List (1993)",
 'Shawshank Redemption, The (1994)',
 'L.A. Confidential (1997)',
 'Titanic (1997)']

In [ ]:
##Movies with lowest bias
idxs = movies_bias.argsort(descending=True)[-5:]
[dls.classes['title'][i] for i in idxs]

['Bloodsport 2 (1995)',
 'Bio-Dome (1996)',
 'Flintstones, The (1994)',
 'Lawnmower Man 2: Beyond Cyberspace (1996)',
 'Children of the Corn: The Gathering (1996)']

In [ ]:
##Using Fastai API
learn = collab_learner(dls, n_factors=50, y_range=(0, 5.5))
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.942344,0.942121,00:12
1,0.871629,0.879534,00:12
2,0.741951,0.838128,00:12
3,0.598942,0.823477,00:12
4,0.494779,0.823641,00:12


In [ ]:
learn.model

EmbeddingDotBias(
  (u_weight): Embedding(944, 50)
  (i_weight): Embedding(1665, 50)
  (u_bias): Embedding(944, 1)
  (i_bias): Embedding(1665, 1)
)

In [ ]:
movie_bias = learn.model.i_bias.weight.squeeze()
idxs = movie_bias.argsort(descending=True)[:5]
[dls.classes["title"][i] for i in idxs]

["Schindler's List (1993)",
 'Titanic (1997)',
 'L.A. Confidential (1997)',
 'Shawshank Redemption, The (1994)',
 'Star Wars (1977)']

In [ ]:
dls.classes['title'].o2i['Silence of the Lambs, The (1991)']

1330

In [ ]:
embs = get_emb_sz(dls)
embs

[(944, 74), (1665, 102)]

In [ ]:
##Using Deep-Learning

class CollabNN(Module):
    def __init__(self, user_sz, item_sz, y_range=(0,5.5), n_act=100):
        self.user_factors = Embedding(*user_sz)
        self.item_factors = Embedding(*item_sz)
        self.layers = nn.Sequential(
            nn.Linear(user_sz[1]+item_sz[1], n_act),
            nn.ReLU(),
            nn.Linear(n_act, 1))
        self.y_range = y_range

    def forward(self, x):
        embs = torch.cat((self.user_factors(x[:,0]), 
                          self.item_factors(x[:,1])), dim=1)
        x = self.layers(embs)
        return sigmoid_range(x, *self.y_range)

In [ ]:
model = CollabNN(*embs)

In [ ]:
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.927294,0.955612,00:14
1,0.926429,0.908407,00:14
2,0.896736,0.886783,00:14
3,0.806315,0.869829,00:14
4,0.799184,0.867705,00:14


In [ ]:
## Using collab learning's api that uses NN
learn = collab_learner(dls, use_nn=True, y_range=(0,5.5), layers=[100,50])
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.953141,0.993890,00:18
1,0.915067,0.906260,00:18
2,0.849275,0.882213,00:18
3,0.799404,0.856484,00:18
4,0.750437,0.856175,00:18
